# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import numpy as np;
import pandas as pd;
from sqlalchemy import create_engine;

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [4]:
# Database connection parameters
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'employees'

# Create the connection
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
query = 'SELECT * FROM employees_advanced'

# Read the data
salaries = pd.read_sql(query, engine)

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below.

In [4]:
salaries.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below.

In [5]:
salaries['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
salaries['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# This is a two-sided test for the null hypothesis that the expected value (mean) of a sample 
# of independent observations is equal to the given population mean.
from scipy.stats import ttest_1samp

In [8]:
# Ho: the mean hourly wage is equal to $30/h.
# Ha: the mean hourly wage is different to $30/h.

alpha = 0.05;
hourly_rate = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
st, p = ttest_1samp(hourly_rate, 30)

print("Null hypothesis rejected") if p < alpha else print("Null hypothesis can't be rejected")

print('Statistic: ', st)
print('p-value: ', p)

Null hypothesis rejected
Statistic:  20.6198057854942
p-value:  4.3230240486229894e-92


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

**Hint**: scipy doesn't have a one-tailed t-test. However, one-tailed tests can be backed out from two-sided tests. A one-tailed test has a p-value that is half of the two-tailed p-value. Given *p* and *statistic* values from a two-tailed test, you would reject the null hypothesis $H_0:$ *less than or equal* when *p/2 < alpha* and *statistic > 0*. For a $H_0:$ *greater than or equal*, you would reject the null hypothesis when *p/2 < alpha* and *statistic < 0*. Try to draw this in a piece of paper to understand it better. 

In [11]:
# TWO TAILED TEST - OPTION 1
# Ho: the salaries this year are equal to last year's mean of $86000/year for salaried employees.
# Ha: the salaries this year are different to last year's mean of $86000/year for salaried employees.

alpha = 0.05;
police_filtered = salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]
police_salaries = police_filtered['Annual Salary']
st, p = ttest_1samp(police_salaries, 86000)

if p < alpha:
    print("Null hypothesis rejected")
    if st < 0:
        print("The mean salary this year is lower than last year's mean")
    else: 
        print("The mean salary this year is greater than last year's mean")
else:
    print("Null hypothesis can't be rejected")
    
print('Statistic: ', st)
print('p-value: ', p)

Null hypothesis rejected
The mean salary this year is greater than last year's mean
Statistic:  3.081997005712994
p-value:  0.0020603403550965137


In [16]:
# ONE TAILED TEST  - OPTION 2
# Ho: the salaries this year are less or equal than last year's mean of $86000/year for salaried employees.
# Ha: the salaries this year are greater than last year's mean of $86000/year for salaried employees.

alpha = 0.05;
police_filtered = salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]
police_salaries = police_filtered['Annual Salary']
st, p_two_tailed = ttest_1samp(police_salaries, 86000)

p_one_tailed = p_two_tailed / 2

if p_one_tailed < alpha and st > 0:
    print("Null hypothesis rejected")
    print("The mean salary this year is greater than last year's mean")
else:
    print("Null hypothesis can't be rejected")
    
print('Statistic: ', st)
print('p-value: ', p_one_tailed)

Null hypothesis rejected
The mean salary this year is greater than last year's mean
Statistic:  3.081997005712994
p-value:  0.0010301701775482569


**Comments**

**What is the difference between the first test and the second test?**
~~~~

It is possible to reject a value in a two tailed test that you would accept 
in a one tailed test. Why? 

Because the two tailed test is just testing if the mean is different: it could 
be different by being EITHER greater or smaller. 

In the one tailed test, we are testing already if it is greater or smaller. 
For that reason, the p-value is half that of the two sided test.
~~~~

Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
pd.crosstab(salaries['Department'], salaries['Salary or Hourly']).sort_values(by = 'Hourly', ascending = False)

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [14]:
# TWO TAILED TEST - OPTION 1
# Ho: the mean hourly wage is equal to $35/hour.
# Ha: the mean hourly wage is different to $35/hour.

alpha = 0.05;
streets_filtered = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN')]
streets_salaries = streets_filtered['Hourly Rate']
st, p = ttest_1samp(streets_salaries, 35)

if p < alpha:
    print("Null hypothesis rejected")
    if st < 0:
        print("The mean hourly wage is lower than $35/hour")
    else: 
        print("The mean hourly wage is greater than $35/hour")
else:
    print("Null hypothesis can't be rejected")
    
print('Statistic: ', st)
print('p-value: ', p)

Null hypothesis rejected
The mean hourly wage is lower than $35/hour
Statistic:  -9.567447887848152
p-value:  3.3378530564707717e-21


In [15]:
# ONE TAILED TEST  - OPTION 2
# Ho: the mean hourly wage is greater or equal than $35/hour.
# Ha: the mean hourly wage is lower than $35/hour.

alpha = 0.05;
streets_filtered = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN')]
streets_salaries = streets_filtered['Hourly Rate']
st, p_two_tailed = ttest_1samp(streets_salaries, 35)

p_one_tailed = p_two_tailed / 2

if p_one_tailed < alpha and st < 0:
    print("Null hypothesis rejected")
    print("The mean hourly wage is lower than $35/hour")
else:
    print("Null hypothesis can't be rejected")
    
print('Statistic: ', st)
print('p-value: ', p_one_tailed)

Null hypothesis rejected
The mean hourly wage is lower than $35/hour
Statistic:  -9.567447887848152
p-value:  1.6689265282353859e-21


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [13]:
from scipy.stats import t, sem

In [14]:
# Variables
conf_lvl = 0.95;
deg_fr = len(hourly_rate) - 1
mean = np.mean(hourly_rate)
sm = sem(hourly_rate)

# Confidence interval
t.interval(conf_lvl, df=deg_fr, loc=mean, scale=sm)

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [15]:
# Variables
conf_lvl = 0.95;
deg_fr = len(police_salaries) - 1
mean = np.mean(police_salaries)
sm = sem(police_salaries)

# Confidence interval
t.interval(conf_lvl, df=deg_fr, loc=mean, scale=sm)

(86177.05631531784, 86795.77269094894)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [16]:
from statsmodels.stats.proportion import proportions_ztest

In [23]:
# Ho: the proportion of hourly workers is equal to 25%.
# Ha: the proportion of hourly workers is different to 25%.

alpha = 0.05;

st, p = proportions_ztest(count=salaries['Salary or Hourly'].value_counts()['Hourly'], nobs=salaries.shape[0], value=0.25)

print("Null hypothesis rejected") if p < alpha else print("Null hypothesis can't be rejected")

print('Statistic: ', st)
print('p-value: ', p)

Null hypothesis rejected
Statistic:  -3.5099964213703005
p-value:  0.0004481127249057967
